In [1]:
import chess.pgn
import sys

f1 = open('white_wins.txt', 'w')
f2 = open('black_wins.txt', 'w')
pgn = open("train.pgn")
n_games = 0

while True:
    game = chess.pgn.read_game(pgn)
    try:
        result = game.headers["Result"]
    except:
        break

    if result == "1-0":
        n_games =+ 1
        board = game.board()
        for move in game.main_line():
            board.push(move)
            f1.write(str(board))
            f1.write("\n- - - - - - - -\n")
        f1.write("\n= = = = = = = =\n")
    if result == "0-1":
        n_games =+ 1
        board = game.board()
        for move in game.main_line():
            board.push(move)
            f2.write(str(board))
            f2.write("\n- - - - - - - -\n")
        f2.write("\n= = = = = = = =\n")

f1.close()        
f2.close()
pgn.close()
print("Done. ", n_games, " games parsed")

Done.  1  games parsed


In [32]:
import tensorflow as tf
white = open('white_wins.txt', 'r')
black = open('black_wins.txt', 'r')



In [33]:
import numpy as np

# convert all data to a list form 
i = 0
boards = []
for line in white:
    temp = line.strip().split(" ")
    boards.append(temp)
# tst = np.load("A00-139_first_10000.npz")
# boards = tst['arr_0']
# targets = tst['arr_1']
# print(boards.shape)
# print(boards[0,1,:,:])
# print(targets.shape)
# print(boards)
boards_data = np.asarray(boards)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [37]:
# print(boards_data.shape)

# First lets create a 8x8 boards
i = 0
boards_final = []
checklist = ['r','n', 'b', 'q', 'k', 'p', 'R','N', 'B', 'Q', 'K', 'P', '.']

for i in range(len(boards_data)):
    if any(check in boards_data[i] for check in checklist):
        boards_final.append(boards_data[i])
    
# print(len(boards_final))
# print(boards_final)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [92]:
# restructure the boards to a chess board shape
boards_np = np.asarray(boards_final)
int(boards_np.shape[0]/8)
boards_np = boards_np.reshape((int(boards_np.shape[0]/8),8,8))

# print(boards_np.shape)

In [93]:
print(boards_np[0])
# convert to a map of 6x8x8 data type with the extra channel for 
# mapping the 6 types of chess pieces
boards_res = []

for i in range(len(boards_np)):
    test = boards_np[i]
    temp1 = np.zeros(test.shape)
    temp2 = np.zeros(test.shape)
    temp3 = np.zeros(test.shape)
    temp4 = np.zeros(test.shape)
    temp5 = np.zeros(test.shape)
    temp6 = np.zeros(test.shape)
    temp = []
    for i in range(len(test)):
        for j in range(len(test[0])):
            # for the rooks
            if test[i][j] == 'r':
                temp1[i][j] = -1
            if test[i][j] == 'R':
                temp1[i][j] = 1

            #for the n's
            if test[i][j] == 'n':
                temp2[i,j] = -1
            if test[i,j] == 'N':
                temp2[i,j] = 1

             #for the bishop
            if test[i][j] == 'b':
                temp3[i,j] = -1
            if test[i,j] == 'B':
                temp3[i,j] = 1

             #for the queen
            if test[i,j] == 'q':
                temp4[i,j] = -1
            if test[i,j] == 'Q':
                temp4[i,j] = 1

             #for the king
            if test[i][j] == 'k':
                temp5[i,j] = -1
            if test[i,j] == 'K':
                temp5[i,j] = 1

             #for the pawns
            if test[i][j] == 'p':
                temp6[i,j] = -1
            if test[i,j] == 'P':
                temp6[i,j] = 1

    temp.append(temp1)
    temp.append(temp2)
    temp.append(temp3)
    temp.append(temp4)
    temp.append(temp5)
    temp.append(temp6)
    temp = np.asarray(temp)
    boards_res.append(temp)

    
boards_res = np.asarray(boards_res)
print(boards_res.shape)
print(boards_res)
# print(temp1)
# print(temp2)
# print(temp3)
# print(temp4)
# print(temp5)
# print(temp6)

[['r' 'n' 'b' 'q' 'k' 'b' 'n' 'r']
 ['p' 'p' 'p' 'p' 'p' 'p' 'p' 'p']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' 'P' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['P' 'P' 'P' 'P' '.' 'P' 'P' 'P']
 ['R' 'N' 'B' 'Q' 'K' 'B' 'N' 'R']]
(216982, 6, 8, 8)
[[[[-1.  0.  0. ...  0.  0. -1.]
   [ 0.  0.  0. ...  0.  0.  0.]
   [ 0.  0.  0. ...  0.  0.  0.]
   ...
   [ 0.  0.  0. ...  0.  0.  0.]
   [ 0.  0.  0. ...  0.  0.  0.]
   [ 1.  0.  0. ...  0.  0.  1.]]

  [[ 0. -1.  0. ...  0. -1.  0.]
   [ 0.  0.  0. ...  0.  0.  0.]
   [ 0.  0.  0. ...  0.  0.  0.]
   ...
   [ 0.  0.  0. ...  0.  0.  0.]
   [ 0.  0.  0. ...  0.  0.  0.]
   [ 0.  1.  0. ...  0.  1.  0.]]

  [[ 0.  0. -1. ... -1.  0.  0.]
   [ 0.  0.  0. ...  0.  0.  0.]
   [ 0.  0.  0. ...  0.  0.  0.]
   ...
   [ 0.  0.  0. ...  0.  0.  0.]
   [ 0.  0.  0. ...  0.  0.  0.]
   [ 0.  0.  1. ...  1.  0.  0.]]

  [[ 0.  0.  0. ...  0.  0.  0.]
   [ 0.  0.  0. ...  0.  0.  0.]
   [ 0.  0.  0